In [22]:
import math
import random
import torch
import torch.nn.functional as F

In [2]:
class Value:
    def __init__(self, data, parents=()):
        self.data = data
        self.grad = 0.0
        self.parents = set(parents)
        self._backward = lambda: None 
    def __repr__(self):
        return f'V = {self.data}'
    def __add__(self, other):
        other = other if isinstance (other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other))
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    def __radd__(self, other):
        return self+other
    def __mul__(self, other):
        other = other if isinstance (other, Value) else Value(other)
        out = Value(self.data*other.data, (self, other))
        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = _backward
        return out
    def __rmul__(self, other):
        return self*other
    def __truediv__(self,other):
        out = self * other**-1
        return out
    def __pow__(self,other):
        assert isinstance(other,(int,float)), 'pow is int/float'
        out = Value(self.data**other, (self,))
        def _backward():
            self.grad += out.grad * other * self.data**(other-1)
        out._backward = _backward
        return out
    def __neg__(self):
        out = self*-1
        return out
    def __sub__(self,other):
        return self+(-other)
    def tanh(self):
        out = Value(math.tanh(self.data),(self,))
        def _backward():
            self.grad += out.grad * (1-out.data**2)
        out._backward = _backward
        return out
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for parent in v.parents:
                    build_topo(parent)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,))
        def _backward():
            self.grad += out.grad * out.data
        out._backward = _backward
        return out
    def log(self):
        out = Value(math.log(self.data), (self,))
        def _backward():
            self.grad += out.grad * 1 / self.data
        out._backward = _backward
        return out

In [3]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    def __call__(self, x):
        prod = sum((xi*wi for xi, wi in zip(x, self.w)),self.b)
        out = prod.tanh() #забыл
        return out
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self,x):
        out = [neuron(x) for neuron in self.neurons]
        out = out[0] if len(out) == 1 else out
        return out
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
class MLP:
    def __init__(self, nin, nouts):
        size = [nin] + nouts
        self.layers = [Layer(size[i], size[i+1]) for i in range (len(nouts))]
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [11]:
n = MLP(3,[4,4,1])
xs = [
  [1.0, 2.0, -1.0],
  [-1.0, 2.0, 0.5],
  [1.5, -2, 0.5],
  [2.0, -0.5, -2.0],
]
ys = [1.0, -1.0, -1.0, 1.0]

In [12]:
# forward, backward, update
for k in range (101):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for yout, ygt in zip(ypred,ys))
    
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    for p in n.parameters():
        p.data += -0.5 * p.grad
    if k%10 == 0:
        print(k, loss)

0 V = 6.2985412587707135
10 V = 0.00033326432216304976
20 V = 0.00013250355048884048
30 V = 9.108770041702595e-05
40 V = 7.289542877361033e-05
50 V = 6.25602169120709e-05
60 V = 5.583784938144848e-05
70 V = 5.1080256846502616e-05
80 V = 4.7512225693580675e-05
90 V = 4.4720499998487795e-05
100 V = 4.2464327617540104e-05


In [13]:
ypred

[V = 0.9972376981286654,
 V = -0.9982967016029137,
 V = -0.9953914938376808,
 V = 0.9967297612454883]

In [24]:
# Exercises

In [14]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [15]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [-3*a**2-0.5*a**(-0.5), 3*cos(3*b)+2.5*b**1.5, c**(-2)] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [16]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
h = 0.000000001
dfda = (f(2+h, 3, 4)-f(2, 3, 4))/h
dfdb = (f(2, 3+h, 4)-f(2, 3, 4))/h
dfdc = (f(2, 3, 4+h)-f(2, 3, 4))/h
numerical_grad = [dfda, dfdb, dfdc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554623700802
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250022721587811


In [17]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h = 0.000000001
dfda = (f(2+h, 3, 4)-f(2-h, 3, 4))/(2*h)
dfdb = (f(2, 3+h, 4)-f(2, 3-h, 4))/(2*h)
dfdc = (f(2, 3, 4+h)-f(2, 3, 4-h))/(2*h)
numerical_grad2 = [dfda, dfdb, dfdc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554623700802
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250022721587811


In [20]:
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [23]:
logits = torch.tensor([0.0, 3.0, -2.0, 1.0]).double() ;  logits.requires_grad = True
probs = F.softmax(logits, 0)
loss = -torch.log(probs[3])
loss.backward()
logits.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)